In [42]:
'''This script imports all the required country data from the WID (World Inequality Database)'''
## Load country names, codes and regions
import os
import pandas as pd

file_path = os.path.join('.', 'data/wid_all_data', 'WID_countries_modified.csv')

if os.path.exists(file_path):
    with open(file_path, encoding='unicode_escape') as f:
        countries = pd.read_csv(f, sep = ";")
else:
    print("File not found. Please check the file path.")

# Remove two commas from the end of each string in the specified column
countries.rename(columns={'region2,,' : 'region2'}, inplace=True)
countries['region2'] = countries['region2'].str.rstrip(',')

# Print the header of the DataFrame
print(countries.head())

  alpha2                 titlename             shortname    region  \
0     AD                   Andorra               Andorra    Europe   
1     AE  the United Arab Emirates  United Arab Emirates      Asia   
2     AF               Afghanistan           Afghanistan      Asia   
3     AG       Antigua and Barbuda   Antigua and Barbuda  Americas   
4     AI                  Anguilla              Anguilla  Americas   

          region2  
0  Western Europe  
1       West Asia  
2      South Asia  
3       Caribbean  
4       Caribbean  


In [43]:
#aptincj992 is pre tax national income per adult (not per adult equivalent I think)
# Define your desired directory
desired_directory = r'C:\Users\y-osw\Dropbox\Arbeit\postdoc_lausanne\convergence\data\wid_all_data'

# Base path for debugging
file_path_to_data = os.path.join('.', 'data\wid_all_data')
print("Base path:", file_path_to_data)
print("Current working directory:", os.getcwd())

# Assuming 'data' is your main DataFrame
# Initialize a dictionary to store the subset dataframes
subset_dfs = {}

# Iterate over the unique codes in the 'alpha2' column
for code in countries['alpha2'].unique():
    # Construct the file name for the respective dataframe
    file_name = f'WID_data_{code}.csv'  # Assuming the files are .csv
    full_file_path = os.path.join(desired_directory, file_name)

    # Check if the file exists
    if os.path.exists(full_file_path):
        # Load the dataframe
        df = pd.read_csv(full_file_path, sep = ";")        
        # Print column names for debugging
        print("Columns in DataFrame:", df.columns)

        # Subset the dataframe based on two variables: 'aptincj992' and 'ntaxad999i'
        # for pre-tax national income distributional accounts and 'year'
        subset_df = df[(df['variable'] == 'aptincj992') & (df['year'] == 2021)]

        # Store the subset dataframe in the dictionary
        subset_dfs[code] = subset_df

        print(subset_df)
        
        # Delete the full dataframe to clear memory
        del df
    else:
        print(f"File not found: {full_file_path}")


# Check if all dataframes have the same format/dimensions
if subset_dfs:  # Check if there's at least one dataframe in the dictionary
    first_df_shape = next(iter(subset_dfs.values())).shape
    consistent_format = all(df.shape == first_df_shape for df in subset_dfs.values())

    if consistent_format:
        print("All dataframes have the same format and dimensions.")
    else:
        print("Dataframes differ in format/dimensions.")
else:
    print("No dataframes were loaded.")

Base path: .\data\wid_all_data
Current working directory: c:\Users\y-osw\Dropbox\Arbeit\postdoc_lausanne\convergence
Columns in DataFrame: Index(['country', 'variable', 'percentile', 'year', 'value', 'age', 'pop'], dtype='object')
Empty DataFrame
Columns: [country, variable, percentile, year, value, age, pop]
Index: []
Columns in DataFrame: Index(['country', 'variable', 'percentile', 'year', 'value', 'age', 'pop'], dtype='object')
      country    variable percentile  year     value  age pop
25401      AE  aptincj992      p0p10  2021    4318.1  992   j
25443      AE  aptincj992     p0p100  2021  222942.6  992   j
25485      AE  aptincj992      p0p50  2021   57102.6  992   j
25527      AE  aptincj992      p0p99  2021  190712.1  992   j
25569      AE  aptincj992    p10p100  2021  247234.2  992   j
...       ...         ...        ...   ...       ...  ...  ..
40435      AE  aptincj992     p67p68  2021  186053.8  992   j
40477      AE  aptincj992     p6p100  2021  237154.6  992   j
40519  

In [44]:
test = subset_dfs['AT']
print(test)

      country    variable percentile  year      value  age pop
54309      AT  aptincj992       p0p1  2021        0.0  992   j
54351      AT  aptincj992      p0p10  2021     1858.1  992   j
54393      AT  aptincj992     p0p100  2021    47801.0  992   j
54435      AT  aptincj992      p0p50  2021    21188.9  992   j
54477      AT  aptincj992      p0p90  2021    34816.4  992   j
...       ...         ...        ...   ...        ...  ...  ..
65271      AT  aptincj992     p9p100  2021    52397.9  992   j
89244      AT  aptincj992    p71p100  2021    97443.7  992   j
89286      AT  aptincj992     p86p87  2021    71166.5  992   j
89328      AT  aptincj992  p99.8p100  2021  1213866.9  992   j
89370      AT  aptincj992      p9p10  2021     6689.0  992   j

[266 rows x 7 columns]


In [45]:
# clear empty dataframes from dictionary    
subset_dfs = {k: v for k, v in subset_dfs.items() if not v.empty}
# adjust dimensions of dictionary

In [46]:
# subset percentiles in each dataframe in the dictionary
# based on column percentile and values 

percentile_values = ['p0p10', 'p10p20', 'p20p30', 'p30p40', 'p40p50', 'p50p60', 'p60p70', 'p70p80', 'p80p90', 
                     'p90p91', 'p91p92', 'p92p93', 'p93p94', 'p94p95', 'p95p96', 'p96p97', 'p97p98', 'p98p99', 
                     'p99p99.1', 'p99.1p99.2', 'p99.2p99.3', 'p99.3p99.4', 'p99.4p99.5', 'p99.5p99.6', 'p99.6p99.7','p99.7p99.8', 'p99.8p99.9', 'p99.9p100']
# Initialize a dictionary to store the subset dataframes
subset_dfs2 = {}

# Iterate all countries in the dictionary and extract the percentiles
for code, df in subset_dfs.items():
    # Subset the dataframe based on 'variable' which is
    #  pre-tax national income distributional accounts
    #  and 'year'
    subset_df = df[df['percentile'].isin(percentile_values)]
    # Store the subset dataframe in the dictionary
    subset_dfs2[code] = subset_df

# Check if all dataframes have the same format/dimensions
if subset_dfs2:  # Check if there's at least one dataframe in the dictionary
    first_df_shape = next(iter(subset_dfs2.values())).shape
    consistent_format = all(df.shape == first_df_shape for df in subset_dfs2.values())

    if consistent_format:
        print("All dataframes have the same format and dimensions.")
    else:
        print("Dataframes differ in format/dimensions.")


# store one dataframe for visual inspection as test2
test2 = subset_dfs2['AT']     

All dataframes have the same format and dimensions.


In [47]:
# make a list of all countries in the dataframe subset_dfs2
countries_list = list(subset_dfs2.keys())

# ALSO store the name of the each country in the subseT_dfs2 dictionary
# in another list and then combine the countries_list and the country_names_list
# into a dictionary
country_names_list = []
for code in countries_list:
    country_names_list.append(countries.loc[countries['alpha2'] == code, 'titlename'].iloc[0])

# combine the two lists into a dictionary
country_dict = dict(zip(countries_list, country_names_list))


In [48]:
# Get the list of countries for which the variable ntaxad992t exists in the original data
countries_with_variable = [code for code, df in subset_dfs.items() if 'ntaxadt992' in df['variable'].values]

# Compare the countries with variable ntaxad992t to the countries in country_dict
countries_with_variable_in_dict = [code for code in countries_with_variable if code in country_dict]

# Print the list of countries
print(countries_with_variable_in_dict)


[]


In [51]:
# check for what countries ntaxadt999 that is the adult population data exists

# Define your desired directory
desired_directory = r'C:\Users\y-osw\Dropbox\Arbeit\postdoc_lausanne\convergence\data\wid_all_data'

# Base path for debugging
file_path_to_data = os.path.join('.', 'data\wid_all_data')
print("Base path:", file_path_to_data)
print("Current working directory:", os.getcwd())

# Initialize a list to store countries where 'ntaxadt992' is present
countries_with_variable = []

# Iterate over the unique codes in the 'alpha2' column
for code in countries['alpha2'].unique():
    # Construct the file name for the respective dataframe
    file_name = f'WID_data_{code}.csv'  # Assuming the files are .csv
    full_file_path = os.path.join(desired_directory, file_name)

    # Check if the file exists
    if os.path.exists(full_file_path):
        # Load the dataframe
        df = pd.read_csv(full_file_path, sep = ";")

        # Check if 'ntaxadt992' exists in this dataframe
        if 'ntaxadt992' in df['variable'].unique():
            # Get the country name (assuming a 'country_name' column exists)
            # country name comes from the countries dataframe and its column short name
            country_name = countries.loc[countries['alpha2'] == code, 'shortname'].iloc[0]  

            # Add the country code and name to the list
            countries_with_variable.append((code, country_name))

# Print the list of countries with 'ntaxadt992'
print("Countries with 'ntaxadt992':", countries_with_variable)

Base path: .\data\wid_all_data
Current working directory: c:\Users\y-osw\Dropbox\Arbeit\postdoc_lausanne\convergence
Countries with 'ntaxadt992': [('AR', 'Argentina'), ('AU', 'Australia'), ('BE', 'Belgium'), ('CA', 'Canada'), ('CO', 'Colombia'), ('DK', 'Denmark'), ('ES', 'Spain'), ('FI', 'Finland'), ('GB', 'United Kingdom'), ('HU', 'Hungary'), ('IN', 'India'), ('IT', 'Italy'), ('JP', 'Japan'), ('KR', 'Korea'), ('MY', 'Malaysia'), ('NO', 'Norway'), ('NZ', 'New Zealand'), ('SE', 'Sweden'), ('SG', 'Singapore'), ('UY', 'Uruguay'), ('ZA', 'South Africa')]
